In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/Shareddrives/Segmentation/

/content/drive/Shareddrives/Segmentation


In [3]:
!nvidia-smi

Sat Jan  9 18:32:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    30W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!pip install einops

In [21]:
import numpy as np
import torch
from torch import nn, einsum
from torch.nn import functional as func
import torch.utils.data as tata
from einops import rearrange
from PIL import Image
import random
import gc
from tqdm import tqdm

In [6]:
class SeparableConv2d(nn.Module):
  def __init__(self,in_channels,out_channels,kernel_size=1,stride=1,padding=0,dilation=1,bias=False):
    super(SeparableConv2d,self).__init__()

    self.conv1 = nn.Conv2d(in_channels,in_channels,kernel_size,stride,padding,dilation,groups=in_channels,bias=bias)
    self.pointwise = nn.Conv2d(in_channels,out_channels,1,1,0,1,1,bias=bias)

  def forward(self,x):
    x = self.conv1(x)
    x = self.pointwise(x)
    return x

In [7]:
class Mish(nn.Module):
  def __init__(self):
    super().__init__()
    # print("Mish activation loaded...")
  def forward(self,x):
    x = x * (torch.tanh(func.softplus(x)))
    return x


In [8]:
""" Parts of the U-Net model """

import torch
import torch.nn as nn
import torch.nn.functional as F


class DoubleConv(nn.Module):
  """(convolution => [BN] => ReLU) * 2"""

  def __init__(self, in_channels, out_channels, mid_channels=None, stride=1, dilation=1):
    super().__init__()
    if not mid_channels:
      mid_channels = out_channels
    self.double_conv = nn.Sequential(
      SeparableConv2d(in_channels, mid_channels, kernel_size=3, padding=dilation, stride=stride, dilation=dilation),
      nn.BatchNorm2d(mid_channels),
      Mish(),
      SeparableConv2d(mid_channels, out_channels, kernel_size=3, padding=dilation, dilation=dilation),
      nn.BatchNorm2d(out_channels),
      Mish()
    )

  def forward(self, x):
    return self.double_conv(x)


class Down(nn.Module):
  """Downscaling with maxpool then double conv"""

  def __init__(self, in_channels, out_channels):
    super().__init__()
    self.maxpool_conv = nn.Sequential(
      nn.MaxPool2d(2,1),
      DoubleConv(in_channels, out_channels, stride=2)
    )

  def forward(self, x):
    return self.maxpool_conv(x)


class Up(nn.Module):
  """Upscaling then double conv"""

  def __init__(self, in_channels, out_channels, bilinear=True):
    super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
    if bilinear:
      self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
      self.conv = ConvLambda(in_channels, out_channels, in_channels // 2)
    else:
      self.up = nn.ConvTranspose2d(in_channels , in_channels // 2, kernel_size=2, stride=2)
      self.conv = ConvLambda(in_channels, out_channels)


  def forward(self, x1, x2):
    x1 = self.up(x1)
    # input is CHW
    diffY = x2.size()[2] - x1.size()[2]
    diffX = x2.size()[3] - x1.size()[3]

    x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
          diffY // 2, diffY - diffY // 2])
    # if you have padding issues, see
    # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
    # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
    x = torch.cat([x2, x1], dim=1)
    return self.conv(x)


class OutConv(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(OutConv, self).__init__()
    self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

  def forward(self, x):
    return self.conv(x)

In [9]:
class DCLoss(nn.Module):
  def __init__(self):
    super(DCLoss, self).__init__()

  def forward(self,pred,label):
    eps = 0.0000001
    a = torch.sum(pred*label, (2, 3))*2
    b = torch.sum(pred+label, (2, 3))
    return 1-torch.mean((a+eps)/(b+eps))

In [10]:
class WeightBCELoss(nn.Module):
  def __init__(self, balance = 0):
    super(WeightBCELoss, self).__init__()
    self.balance = balance

  def forward(self,pred,label):
    b,c,h,w = pred.shape
    eps = 0.0000001
    weight = 1 - torch.sum(pred, (2, 3),keepdim=True)/(h*w)
    weight = (weight + self.balance)/(1 + 2*self.balance)
    pred = torch.clip(pred, eps, 1-eps)
    true_loss = weight * label * torch.log(pred)
    false_loss = (1-weight) * (1-label) * torch.log(1-pred)
    return -torch.mean(true_loss + false_loss)

In [11]:
""" Full assembly of the parts to form the complete network """

import torch.nn.functional as F


class Net(nn.Module):
  def __init__(self, n_channels, n_classes, bilinear=True):
    super(Net, self).__init__()
    self.n_channels = n_channels
    self.n_classes = n_classes
    self.bilinear = bilinear

    self.inc = DoubleConv(n_channels, 32)
    self.down1 = Down(32, 64)
    self.down2 = Down(64, 128)
    self.down3 = Down(128, 256)
    self.down4 = Down(256, 512)

    self.conv1 = DoubleConv(120, 32, dilation=1)
    self.conv2 = DoubleConv(32, 32, dilation=1)
    self.conv3 = DoubleConv(32, 32, dilation=1)
    self.conv4 = DoubleConv(32, 32, dilation=1)
    self.conv5 = DoubleConv(128, 128, dilation=1)
    self.conv6 = DoubleConv(32, 32, dilation=1)
    self.outc = OutConv(32, n_classes)

  def forward(self, x):
    x1 = self.inc(x)
    x2 = self.down1(x1)
    x3 = self.down2(x2)
    x4 = self.down3(x3)
    x5 = self.down4(x4)

    x = torch.cat([
      x2,
      func.pixel_shuffle(x3, 2),
      func.pixel_shuffle(x4, 4),
      func.pixel_shuffle(x5, 8),
    ],1)

    d1 = self.conv1(x)
    d2 = self.conv2(d1)
    d3 = self.conv3(d2)
    d4 = self.conv4(d3)

    x = torch.cat([
      d1,
      d2,
      d3,
      d4,
    ],1)
    d5 = self.conv5(x)
    d5 = func.pixel_shuffle(d5, 2)
    d6 = self.conv6(d5 + x1)
    logits = self.outc(d6)
    return torch.sigmoid(logits)

In [12]:
# loss_MSE = nn.MSELoss()
loss_DC = DCLoss()
# loss_BCE = nn.BCELoss()
loss_WeightBCE = WeightBCELoss(0.2)

In [13]:
epoch = 5
batch_size = 16

In [14]:
def test(data,model):
  with torch.no_grad():
    data_indices = [i for i in range(data["MR"].shape[0])]
    random.shuffle(data_indices)
    batch_num = len(data_indices)//batch_size
    loss_CT = 0
    loss_FT = 0
    loss_MN = 0
    for batch_idx in range(batch_num):
      MR = data["MR"][data_indices[batch_idx*batch_size:(batch_idx+1)*batch_size]]
      MR = torch.tensor(MR/255,dtype=torch.float32).cuda()
      label = data["label"][data_indices[batch_idx*batch_size:(batch_idx+1)*batch_size]]
      label = torch.tensor(label/255,dtype=torch.float32).cuda()
      
      out = model(MR)
      out = out>=0.5
      pred_CT, pred_FT, pred_MN = torch.split(out,1,1)
      label_CT, label_FT, label_MN = torch.split(label,1,1)
      loss_CT += loss_DC(pred_CT, label_CT)*(batch_size/len(data_indices))
      loss_FT += loss_DC(pred_FT, label_FT)*(batch_size/len(data_indices))
      loss_MN += loss_DC(pred_MN, label_MN)*(batch_size/len(data_indices))
    if batch_num * batch_size != len(data_indices):
      MR = data["MR"][data_indices[batch_num*batch_size:]]
      MR = torch.tensor(MR/255,dtype=torch.float32).cuda()
      label = data["label"][data_indices[batch_num*batch_size:]]
      label = torch.tensor(label/255,dtype=torch.float32).cuda()
      
      out = model(MR)
      out = out>=0.5
      pred_CT, pred_FT, pred_MN = torch.split(out,1,1)
      label_CT, label_FT, label_MN = torch.split(label,1,1)
      loss_CT += loss_DC(pred_CT, label_CT)*(1 - (batch_num * batch_size/len(data_indices)))
      loss_FT += loss_DC(pred_FT, label_FT)*(1 - (batch_num * batch_size/len(data_indices)))
      loss_MN += loss_DC(pred_MN, label_MN)*(1 - (batch_num * batch_size/len(data_indices)))
    print((1-(loss_CT+loss_FT+loss_MN)/3).cpu())
    print((1-loss_CT).cpu())
    print((1-loss_FT).cpu())
    print((1-loss_MN).cpu())

In [15]:
def train(data,model,opt):
  data_indices = [i for i in range(data["MR"].shape[0])]
  random.shuffle(data_indices)
  batch_num = len(data_indices)//batch_size
  for batch_idx in range(batch_num):
    MR = data["MR"][data_indices[batch_idx*batch_size:(batch_idx+1)*batch_size]]
    MR = torch.tensor(MR/255,dtype=torch.float32).cuda()
    label = data["label"][data_indices[batch_idx*batch_size:(batch_idx+1)*batch_size]]
    label = torch.tensor(label/255,dtype=torch.float32).cuda()
    opt.zero_grad()
    
    out = model(MR)
    loss = loss_WeightBCE(out, label)*0.3
    loss += loss_DC(out, label)*0.7
    # loss += loss_MSE(out, label)
    # loss += loss_BCE(out, label)
    # # loss += loss_ssim(out, label)
    # loss /= 2

    loss.backward()
    opt.step()
    gc.collect()
    torch.cuda.empty_cache()
  if batch_num * batch_size != len(data_indices):
    MR = data["MR"][data_indices[batch_num*batch_size:]]
    MR = torch.tensor(MR/255,dtype=torch.float32).cuda()
    label = data["label"][data_indices[batch_num*batch_size:]]
    label = torch.tensor(label/255,dtype=torch.float32).cuda()
    opt.zero_grad()
    
    out = model(MR)
    loss = loss_WeightBCE(out, label)*0.3
    loss += loss_DC(out, label)*0.7
    # loss += loss_MSE(out, label)
    # loss += loss_BCE(out, label)
    # # loss += loss_ssim(out, label)

    loss.backward()
    opt.step()
    gc.collect()
    torch.cuda.empty_cache()

In [36]:
for i in range(10):
  print(i)
  data = np.load("./carpalTunnel/{}/data.npz".format(i))

0
1
2
3
4
5
6
7
8
9


In [37]:
for t in range(5):
  train_files = [j for j in range(0,t*2)]+[j for j in range((t+1)*2,10)]
  test_files = [j for j in range(t*2,(t+1)*2)]
  netModel = Net(n_channels=2,n_classes=3).cuda()
  opt = torch.optim.Adamax(netModel.parameters())
  opt.step()
  gc.collect()
  torch.cuda.empty_cache()
  for i in range(epoch):
    train_file_indices = [*train_files]
    random.shuffle(train_file_indices)
    for file_idx in tqdm(train_file_indices):
      data = np.load("./carpalTunnel/{}/data.npz".format(file_idx))
      train(data,netModel,opt)
    for file_idx in test_files:
      data = np.load("./carpalTunnel/{}/data.npz".format(file_idx))
      test(data,netModel)
  torch.save(netModel.state_dict(), "netModel-cross-{}.pkl".format(t))
    




  0%|          | 0/8 [00:00<?, ?it/s]

 12%|█▎        | 1/8 [00:44<05:08, 44.08s/it]

 25%|██▌       | 2/8 [01:29<04:27, 44.63s/it]

 38%|███▊      | 3/8 [02:07<03:31, 42.35s/it]

 50%|█████     | 4/8 [02:58<03:00, 45.13s/it]

 62%|██████▎   | 5/8 [03:45<02:16, 45.56s/it]

 75%|███████▌  | 6/8 [04:25<01:27, 43.89s/it]

 88%|████████▊ | 7/8 [05:10<00:44, 44.20s/it]

100%|██████████| 8/8 [05:45<00:00, 43.13s/it]


tensor(0.7166)
tensor(0.8154)
tensor(0.8525)
tensor(0.4818)




  0%|          | 0/8 [00:00<?, ?it/s]

tensor(0.6673)
tensor(0.8448)
tensor(0.7918)
tensor(0.3653)




 12%|█▎        | 1/8 [00:39<04:37, 39.65s/it]

 25%|██▌       | 2/8 [01:27<04:12, 42.09s/it]

 38%|███▊      | 3/8 [02:13<03:36, 43.23s/it]

 50%|█████     | 4/8 [03:04<03:02, 45.64s/it]

 62%|██████▎   | 5/8 [03:59<02:24, 48.30s/it]

 75%|███████▌  | 6/8 [04:44<01:35, 47.50s/it]

 88%|████████▊ | 7/8 [05:27<00:45, 45.98s/it]

100%|██████████| 8/8 [06:04<00:00, 45.52s/it]


...

tensor(0.8089)
tensor(0.8995)
tensor(0.8826)
tensor(0.6447)




  0%|          | 0/8 [00:00<?, ?it/s]

tensor(0.7453)
tensor(0.9268)
tensor(0.8678)
tensor(0.4413)




 12%|█▎        | 1/8 [00:38<04:31, 38.75s/it]

 25%|██▌       | 2/8 [01:23<04:02, 40.43s/it]

 38%|███▊      | 3/8 [02:04<03:24, 40.80s/it]

 50%|█████     | 4/8 [02:54<02:54, 43.53s/it]

 62%|██████▎   | 5/8 [03:36<02:08, 42.96s/it]

 75%|███████▌  | 6/8 [04:15<01:23, 41.70s/it]

 88%|████████▊ | 7/8 [04:51<00:40, 40.04s/it]

100%|██████████| 8/8 [05:32<00:00, 41.61s/it]


tensor(0.8147)
tensor(0.8868)
tensor(0.8721)
tensor(0.6853)
tensor(0.7705)
tensor(0.9051)
tensor(0.8487)
tensor(0.5579)
